In [1]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import string
import warnings
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from scipy.spatial import distance
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
import nltk.translate.bleu_score as bleu_score
from pyvi import ViTokenizer, ViPosTagger
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('chatbot.csv', encoding='utf-8')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,Unnamed: 0,user_a,user_b
0,0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,1,Có crush ai không,Có 1 bạn cùng lớp
2,2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,3,Có hay nói chuyện không,Hay nhắn tin messenger
4,4,Bạn kia có bắt chuyện trước không,Có đôi khi


In [19]:
def sentance(x):
  return ' '.join([a for a in (''.join([a for a in x if a not in string.punctuation])).lower().split()])

def highestMatch_BleuScore(a):
  temp = {}
  for i in range(len(df["user_a"])):
    #print(df["user_b"][i])
    b = [sentance(a).split()]
    c = sentance(df["user_a"][i]).split()
    temp.update({bleu_score.sentence_bleu(b,c): df["user_b"][i]})
    #temp([bleu_score.sentence_bleu(sentance(a), sentance(data))],[sentance(data)])
  
  temp = sorted(temp.items(), key=lambda item: item[0], reverse=True)
  #print(temp)
  return temp

def bleuScore(a):
  a = a.lower()
  result = highestMatch_BleuScore(a)[0][1]
  return result

In [20]:
print(bleuScore('bạn xem phim ở nhà hay ra rạp ?'))
print(bleuScore('trường đó ở đâu?'))
print(bleuScore('tên bạn là gì'))
print(bleuScore('quê bạn ở đâu?'))
print(bleuScore('bạn có chơi game không?'))
print(bleuScore('bạn là sinh viên năm mấy ?'))
print(bleuScore('bạn đã đi làm chưa?'))
print(bleuScore('bạn hay đi chơi ở đâu?'))

CGV , Galaxy ...
trường đó ở Sài Gòn
Lợi
ở vĩnh long
có
giống mày
Rồi
đi coi phim


In [27]:
sents=[]
for i in range(len(df["user_a"])):
  tokens = ViTokenizer.tokenize(df["user_a"][i])
  sents.append(tokens)

modelw2v = Word2Vec(sentences=sents, size=100, window=5, 
                    sg=1, min_count=2)

In [28]:
def avg_sentence_vector(words, model, num_features, index2word_set):
  featureVec = np.zeros((num_features,), dtype="float32")
  nwords = 0
  
  for word in words:
    if word in index2word_set:
      nwords = nwords+1
      featureVec = np.add(featureVec, model.wv[word])

  if nwords>0:
    featureVec = np.divide(featureVec, nwords)
  return featureVec

In [50]:
def W2V(s):
  sentence_1_avg_vector = avg_sentence_vector(s, model=modelw2v, num_features=100, index2word_set= modelw2v.wv.index2word)
  #print(sentence_1_avg_vector)
  max_ans=1
  for sentence_2 in df["user_a"]:
    #print(sentence_2)
    sentence_2_avg_vector = avg_sentence_vector(sentence_2, model=modelw2v, num_features=100, index2word_set= modelw2v.wv.index2word)

    values =  distance.euclidean(sentence_1_avg_vector,sentence_2_avg_vector)
    #print("values",values)
    if(values<max_ans):
      max_ans=values
      #print("values",values)
      #print("sentence_2",sentence_2)
      result = df.loc[df["user_a"] == sentence_2]["user_b"]
      result = result.iloc[0]
  return result

In [52]:
print(W2V('bạn xem phim ở nhà hay ra rạp ?'))
print(W2V('trường đó ở đâu?'))
print(W2V('tên bạn là gì'))
print(W2V('quê bạn ở đâu?'))
print(W2V('bạn có chơi game không?'))
print(W2V('bạn là sinh viên năm mấy ?'))
print(W2V('bạn đã đi làm chưa?'))
print(W2V('bạn hay đi chơi ở đâu?'))

nhà
trường đó ở Sài Gòn
Lợi
hưng yên
Có nhé
Mình sinh viên năm 3
Rồi
Bà Nà Hill
